In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [37]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short short on {topic} with {number} box office collection"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [38]:
chain = prompt | model | output_parser

In [39]:
chain.invoke({"topic": "bahubali","number":"3"})

"Bahubali is an epic Indian film that follows the story of a young man named Shivudu, who discovers his royal heritage and embarks on a quest to avenge his father's death and reclaim his rightful place as the king of Mahishmati. Along the way, he falls in love with the beautiful warrior princess Avantika and faces off against the tyrannical Bhallaladeva.\n\nThe film was a massive success at the box office, earning a staggering $100 million in its opening weekend alone. It went on to become the highest-grossing Indian film of all time, with a total worldwide box office collection of over $250 million. The film's stunning visuals, epic battles, and emotional storyline captivated audiences around the world, solidifying its place as a modern cinematic masterpiece."

In [23]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [27]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["founder of amazon is jeff bezos", "founder of spacex and tesla is elon musk"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [28]:
retriever.get_relevant_documents("who is the founder of tesla?")

[Document(page_content='founder of spacex and tesla is elon musk'),
 Document(page_content='founder of amazon is jeff bezos')]

In [30]:
retriever.get_relevant_documents("who is founder Amazon?")

[Document(page_content='founder of amazon is jeff bezos'),
 Document(page_content='founder of spacex and tesla is elon musk')]

In [31]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [32]:
from langchain.schema.runnable import RunnableMap

In [33]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [34]:
chain.invoke({"question": "who is the founder of spacex?"})

'Elon Musk'

In [35]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [36]:
inputs.invoke({"question": "who is jeff bezos?"})

{'context': [Document(page_content='founder of amazon is jeff bezos'),
  Document(page_content='founder of spacex and tesla is elon musk')],
 'question': 'who is jeff bezos?'}